In [2]:
import pandas as pd
import json

# Load the JSONL file
file_path = 'HedgePeer.jsonl'  # Replace with the actual file path

data = []
with open(file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        for sentence in item['Sentences']:
            certain = 0 if sentence['Hedges'] else 1
            data.append({
                'Sentence': sentence['Sentence'],
                'Certain': certain
            })

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


                                                Sentence  Certain
0      " The idea of using cross task transfer perfor...        1
1      Please refer to the paper Discovering structur...        1
2      One issue of the use of cross task transfer pe...        1
3      For example , in binary classification tasks ,...        0
4      So the use of cross task transfer performance ...        1
...                                                  ...      ...
55044  Any reason why the authors did not try on othe...        1
55045  As a result I feel that the title and the clai...        0
55046  Over all the paper is clearly written and easy...        1
55047  Based on its incremental nature and weak exper...        1
55048  Happy to change my opinion if other reviewers ...        0

[55049 rows x 2 columns]


In [3]:
!pip install -U transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [4]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from transformers import DataCollatorWithPadding

In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset, DatasetDict

# Rename columns if necessary
df.rename(columns={'Sentence': 'text', 'Certain': 'labels'}, inplace=True)

# Split the DataFrame into train and test sets
train_df, val_test_df = train_test_split(df, test_size=0.6, random_state=42)

# Further split the validation and test sets
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

# Convert DataFrames to Datasets# Reset the index of your DataFrame
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})


In [6]:
model_path = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_path)
id2label = {0: "Uncertain", 1: "Certain"}
label2id = {"Uncertain": 0, "Certain": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, id2label=id2label, label2id=label2id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# freeze base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_data = dataset_dict.map(preprocess_function, batched=True)


Map:   0%|          | 0/22019 [00:00<?, ? examples/s]

Map:   0%|          | 0/16515 [00:00<?, ? examples/s]

Map:   0%|          | 0/16515 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [10]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

In [ ]:
"""lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-uncertainty-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
"""

<ipython-input-12-cdd51a082eee>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.404300,0.416462,0.804000,0.876000
2,0.375700,0.334025,0.857000,0.886000
3,0.366200,0.334819,0.857000,0.892000
4,0.361000,0.323841,0.860000,0.894000
5,0.354700,0.338387,0.852000,0.892000
6,0.354800,0.322107,0.861000,0.897000
7,0.351300,0.316440,0.864000,0.900000
8,0.345500,0.315609,0.866000,0.900000
9,0.337600,0.315742,0.865000,0.900000
10,0.337100,0.314544,0.866000,0.901000


TrainOutput(global_step=27530, training_loss=0.35881143910123614, metrics={'train_runtime': 1470.4706, 'train_samples_per_second': 149.741, 'train_steps_per_second': 18.722, 'total_flos': 6362792040664560.0, 'train_loss': 0.35881143910123614, 'epoch': 10.0})

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Hyperparameters to test
learning_rates = [2e-4, 2e-3]  # Small, base, large learning rates
batch_sizes = [16, 32]  # Base and larger batch sizes
epochs_list = [3]  # start with 3, and choose the best to run 5 or 10

# Define training arguments
training_args = TrainingArguments(
    output_dir="bert-uncertainty-classifier_teacher",
    learning_rate=2e-4,  # Will be overridden per experiment
    per_device_train_batch_size=8,  # Will be overridden
    per_device_eval_batch_size=8,  # Will be overridden
    num_train_epochs=10,  # Will be overridden
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="Accuracy",  # Assuming compute_metrics returns accuracy
    greater_is_better=True,
    report_to="none",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from itertools import product
import heapq

top_models = []  # Store (accuracy, model, (lr, batch_size, num_epochs))

for lr, batch_size, num_epochs in product(learning_rates, batch_sizes, epochs_list):
    print(f"Training with lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")

    training_args.learning_rate = lr
    training_args.per_device_train_batch_size = batch_size
    training_args.per_device_eval_batch_size = batch_size
    training_args.num_train_epochs = num_epochs

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    trainer.train()

    # Evaluate model
    metrics = trainer.evaluate()
    print(metrics)
    current_accuracy = metrics.get("Accuracy", 0.0)

    # Store the model, keeping only the top 3
    heapq.heappush(top_models, (current_accuracy, model, (lr, batch_size, num_epochs)))

    # Keep only the top 3 models
    if len(top_models) > 3:
        heapq.heappop(top_models)

# Sort the models from highest to lowest accuracy
top_models.sort(reverse=True, key=lambda x: x[0])

# Print the top 3 models
print("\nTop 3 Models:")
for rank, (accuracy, model, params) in enumerate(top_models, start=1):
    print(f"{rank}. lr={params[0]}, batch_size={params[1]}, num_epochs={params[2]} with accuracy={accuracy}")


Training with lr=0.0002, batch_size=16, num_epochs=3


<ipython-input-33-784eb913ec10>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.342700,0.327570,0.858000,0.898000
2,0.335400,0.311697,0.867000,0.902000
3,0.336000,0.310809,0.868000,0.902000


Training with lr=0.0002, batch_size=32, num_epochs=3


<ipython-input-33-784eb913ec10>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.425700,0.361515,0.835000,0.882000
2,0.391600,0.325688,0.860000,0.895000
3,0.356500,0.316755,0.866000,0.898000


Training with lr=0.002, batch_size=32, num_epochs=3


<ipython-input-33-784eb913ec10>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.387800,0.365468,0.831000,0.886000
2,0.365600,0.317875,0.863000,0.897000
3,0.338800,0.310250,0.867000,0.904000



Top 3 Models:
1. lr=0.0002, batch_size=32, num_epochs=3 with accuracy=0.0
2. lr=0.002, batch_size=32, num_epochs=3 with accuracy=0.0
3. lr=0.002, batch_size=16, num_epochs=3 with accuracy=0.0


In [ ]:
#accuracy did not properly sort, but I see the top 3 accuracy:

# it is the model with lr = 0.0002 and batch_size = 32
# second is model with lr = 0.0002 and batch_size = 16
# third is model with lr = 0.002 and batch_size = 32

In [ ]:
# Sort the models from highest to lowest accuracy
top_models.sort(reverse=True, key=lambda x: x[0])

# Print the top 3 models
print("\nTop 3 Models:")
for rank, (accuracy, model, params) in enumerate(top_models, start=1):
    print(f"{rank}. lr={params[0]}, batch_size={params[1]}, num_epochs={params[2]} with accuracy={accuracy}")


Top 3 Models:
1. lr=0.0002, batch_size=32, num_epochs=3 with accuracy=0.0
2. lr=0.002, batch_size=32, num_epochs=3 with accuracy=0.0
3. lr=0.002, batch_size=16, num_epochs=3 with accuracy=0.0


In [ ]:
#All models performed similarly, so I will just try 5 and 10 epochs on the top accuracy and choose the best model between those (This method is heuristic but we did this on the homework)

In [11]:
lr = 2e-4
batch_size = 32
num_epochs = 5

training_args = TrainingArguments(
    output_dir="bert-uncertainty-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-11-a5142cd46ac0>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.398300,0.361309,0.841000,0.874000
2,0.362400,0.337538,0.855000,0.883000
3,0.351900,0.328454,0.856000,0.892000
4,0.342900,0.324427,0.862000,0.895000
5,0.341600,0.320855,0.863000,0.895000


TrainOutput(global_step=3445, training_loss=0.3594192615614228, metrics={'train_runtime': 924.8727, 'train_samples_per_second': 119.038, 'train_steps_per_second': 3.725, 'total_flos': 4635746489944920.0, 'train_loss': 0.3594192615614228, 'epoch': 5.0})

In [12]:
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.861), 'AUC': np.float64(0.892)}


In [13]:
lr = 2e-4
batch_size = 32
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-uncertainty-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-13-a1caeb973648>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.346600,0.333944,0.856000,0.891000
2,0.344500,0.323907,0.862000,0.892000
3,0.343200,0.335045,0.850000,0.898000
4,0.339300,0.321548,0.863000,0.900000
5,0.341800,0.322322,0.862000,0.897000
6,0.342400,0.315984,0.862000,0.901000
7,0.337300,0.314013,0.866000,0.901000
8,0.337200,0.316069,0.866000,0.902000
9,0.336600,0.315498,0.865000,0.901000
10,0.334300,0.312029,0.868000,0.901000


TrainOutput(global_step=6890, training_loss=0.3403125602378901, metrics={'train_runtime': 1868.5138, 'train_samples_per_second': 117.842, 'train_steps_per_second': 3.687, 'total_flos': 9252832650510480.0, 'train_loss': 0.3403125602378901, 'epoch': 10.0})

In [14]:
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.864), 'AUC': np.float64(0.898)}


In [ ]:
#Best model is: lr = 0.0002 and batch_size = 32 and epochs = 10



In [ ]:
import torch

def predict_sentence(sentence, model, tokenizer):
    # Detect the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the detected device
    model.to(device)

    # Tokenize the input sentence and move it to the same device as the model
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True).to(device)

    # Ensure model is in evaluation mode
    model.eval()

    # Get the model's output logits
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits

    # Get the predicted class (0 or 1)
    predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class

# Example usage
sentence = "Photosynthesis is a force, but maybe it isn't."
prediction = predict_sentence(sentence, model, tokenizer)
print("Predicted Class:", prediction)  # Output: 0 for Uncertain, 1 for Certain


In [ ]:
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

AttributeError: 'list' object has no attribute 'keys'

In [20]:
!pip install -U transformers huggingface_hub

In [21]:
from huggingface_hub import notebook_login
notebook_login()  # Follow the token prompt

In [26]:
# Correct push_to_hub syntax with explicit parameters
trainer.push_to_hub()


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sdornala/bert-uncertainty-classifier_teacher/commit/91b3ccedf49b71d09cf218a4c1e2292a751fbd84', commit_message='End of training', commit_description='', oid='91b3ccedf49b71d09cf218a4c1e2292a751fbd84', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sdornala/bert-uncertainty-classifier_teacher', endpoint='https://huggingface.co', repo_type='model', repo_id='sdornala/bert-uncertainty-classifier_teacher'), pr_revision=None, pr_num=None)